### I — Ingest

##### Goal 

Load raw accel & gyro files, keep everything as UTC time, and prepare to expand batched samples (Garmin usually stores a base timestamp_ms plus list columns for offsets and x/y/z values).


In [ ]:
# ---- CONFIG ----
CONFIG = {
    "accel_path": "2025-03-23-15-23-10-accelerometer_data.xlsx",
    "gyro_path":  "2025-03-23-15-23-10-gyroscope_data.xlsx",
    "target_hz":  50,         # target sampling rate
    "merge_tolerance_ms": 1,  # max allowed accel↔gyro timestamp gap for alignment
    "tz": "UTC",              # keep everything in UTC
}


## 

## 2) Imports

In [1]:
import json
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_dt
from datetime import timezone, datetime, timedelta


## 3) Load raw Excel files (no parsing yet)

In [3]:
accel_raw = pd.read_excel("2025-03-23-15-23-10-accelerometer_data.xlsx")
gyro_raw  = pd.read_excel("2025-03-23-15-23-10-gyroscope_data.xlsx")

print("Accel shape:", accel_raw.shape)
print("Gyro  shape:", gyro_raw.shape)
accel_raw.head(3), gyro_raw.head(3)


Accel shape: (14536, 6)
Gyro  shape: (14536, 6)


(                 timestamp  timestamp_ms  \
 0  03/24/2025, 07:52:19 AM           766   
 1  03/24/2025, 07:52:20 AM            23   
 2  03/24/2025, 07:52:20 AM           271   
 
                                   sample_time_offset  \
 0  ["0","10","29","38","48","57","66","77","85","...   
 1  ["0","9","19","28","38","48","57","67","77","8...   
 2  ["0","10","19","29","38","47","58","66","76","...   
 
                                   calibrated_accel_x  \
 0  ["-62.48186","-27.60828","-95.41802","-155.478...   
 1  ["-178.7271","-161.2903","-133.1977","-124.479...   
 2  ["-81.85607","-92.51189","-101.2303","-112.854...   
 
                                   calibrated_accel_y  \
 0  ["-758.6971","-984.1958","-1240.532","-1338.82...   
 1  ["-1360.991","-1342.681","-1269.442","-1171.14...   
 2  ["-1140.310","-1056.471","-967.8134","-805.917...   
 
                                   calibrated_accel_z  
 0  ["-1097.585","-1221.784","-1086.742","-957.614...  
 1  ["-777.2302"

## P — Process
#### Goal 

Turn batched rows into one row per sample, compute absolute timestamps for each sub‑sample, align accel & gyro, resample to exactly 50 Hz, and standardize column names/units.

##### We’ll assume typical Garmin columns:

timestamp_ms (int, base time per row),

sample_time_offset (list[int] ms),

x, y, z (each list[float] per row).
If your column names differ, update the mapping once in the code block below.

### 4) Helpers for exploding batched rows

In [5]:
# A function that converts each batched row into per‑sample rows with absolute timestamps
def explode_sensor(df, ts_col="timestamp_ms", off_col="sample_time_offset",
                   x_col="x", y_col="y", z_col="z", prefix="A"):
    """
    df: raw sensor table with base timestamp + lists of offsets and x/y/z.
    prefix: 'A' for accel, 'G' for gyro (used to prefix columns in output).
    """
    # Ensure list-like cols are truly Python lists
    for c in [off_col, x_col, y_col, z_col]:
        if df[c].dtype == object:
            # If the cell is a JSON string, parse it; if it’s already list, keep it
            df[c] = df[c].apply(lambda v: v if isinstance(v, (list, tuple, np.ndarray)) 
                                else json.loads(v) if isinstance(v, str) else v)

    # Length sanity: offsets, x, y, z must be same length per row
    def _check_lengths(row):
        lens = list(map(len, [row[off_col], row[x_col], row[y_col], row[z_col]]))
        return len(set(lens)) == 1

    bad = df[~df.apply(_check_lengths, axis=1)]
    if len(bad):
        print(f"[WARN] {len(bad)} rows with mismatched list lengths. They will be dropped.")
        df = df[df.apply(_check_lengths, axis=1)].copy()

    # Build long-form rows
    rows = []
    for _, r in df.iterrows():
        base = int(r[ts_col])
        offs = r[off_col]
        xs, ys, zs = r[x_col], r[y_col], r[z_col]
        for o, xv, yv, zv in zip(offs, xs, ys, zs):
            abs_ms = base + int(o)
            rows.append({
                "timestamp_ms": abs_ms,
                f"{prefix}x": float(xv),
                f"{prefix}y": float(yv),
                f"{prefix}z": float(zv),
            })
    out = pd.DataFrame(rows).sort_values("timestamp_ms").reset_index(drop=True)
    # to datetime (UTC)
    out["timestamp"] = pd.to_datetime(out["timestamp_ms"], unit="ms", utc=True)
    return out[["timestamp", "timestamp_ms", f"{prefix}x", f"{prefix}y", f"{prefix}z"]]


## 5) Expand accel & gyro to per‑sample rows

In [20]:
# Option A: rename once, keep your existing explode_sensor(..., x_col="x", ...)
accel = accel_raw.rename(columns={
    "calibrated_accel_x":"x", "calibrated_accel_y":"y", "calibrated_accel_z":"z"
})
gyro  = gyro_raw.rename(columns={
    "calibrated_gyro_x":"x", "calibrated_gyro_y":"y", "calibrated_gyro_z":"z"
})



In [21]:
accel.head(3), gyro.head(3)

(                 timestamp  timestamp_ms  \
 0  03/24/2025, 07:52:19 AM           766   
 1  03/24/2025, 07:52:20 AM            23   
 2  03/24/2025, 07:52:20 AM           271   
 
                                   sample_time_offset  \
 0  [0, 10, 29, 38, 48, 57, 66, 77, 85, 95, 104, 1...   
 1  [0, 9, 19, 28, 38, 48, 57, 67, 77, 86, 96, 105...   
 2  [0, 10, 19, 29, 38, 47, 58, 66, 76, 86, 95, 10...   
 
                                                    x  \
 0  ["-62.48186","-27.60828","-95.41802","-155.478...   
 1  ["-178.7271","-161.2903","-133.1977","-124.479...   
 2  ["-81.85607","-92.51189","-101.2303","-112.854...   
 
                                                    y  \
 0  ["-758.6971","-984.1958","-1240.532","-1338.82...   
 1  ["-1360.991","-1342.681","-1269.442","-1171.14...   
 2  ["-1140.310","-1056.471","-967.8134","-805.917...   
 
                                                    z  
 0  ["-1097.585","-1221.784","-1086.742","-957.614...  
 1  ["-777.2302"

## 6) Align accelerometer & gyroscope (nearest timestamp with tolerance)


We align accel and gyro by the nearest timestamp using merge_asof, with a tolerance so mismatched samples are skipped. We also compute diagnostics: the match ratio and the alignment lag distribution in milliseconds.

In [23]:
# ---- edit here if needed ----
MERGE_TOLERANCE_MS = 1
# -----------------------------

tol = pd.Timedelta(milliseconds=MERGE_TOLERANCE_MS)

# Keep a copy of gyro ts to compute lag
gyro2 = gyro.rename(columns={"timestamp":"gyro_ts"}).copy()

merged = pd.merge_asof(
    accel.sort_values("timestamp"), 
    gyro2.sort_values("gyro_ts"),
    left_on="timestamp", right_on="gyro_ts",
    direction="nearest", tolerance=tol
)

have_gyro = merged[["Gx","Gy","Gz"]].notna().all(axis=1)
match_ratio = float(have_gyro.mean())
delta_ms = ((merged.loc[have_gyro, "timestamp"].view("int64") - merged.loc[have_gyro, "gyro_ts"].view("int64")) / 1e6)

print(f"[Step 6] Matched within {MERGE_TOLERANCE_MS} ms: {match_ratio:.1%}")
if len(delta_ms):
    print(f"[Step 6] Lag(ms): mean={delta_ms.mean():.3f}, std={delta_ms.std():.3f}, max_abs={np.abs(delta_ms).max():.3f}")
else:
    print("[Step 6] No matched rows. Try a larger MERGE_TOLERANCE_MS.")

merged = merged.loc[have_gyro, ["timestamp","Ax","Ay","Az","Gx","Gy","Gz"]].reset_index(drop=True)
print("[Step 6] merged shape:", merged.shape)
merged.head(3)


[Step 6] Matched within 1 ms: 100.0%
[Step 6] Lag(ms): mean=0.000, std=0.000, max_abs=0.000
[Step 6] merged shape: (14536, 7)


C:\Users\shali\AppData\Local\Temp\ipykernel_68520\4049600805.py:19: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  delta_ms = ((merged.loc[have_gyro, "timestamp"].view("int64") - merged.loc[have_gyro, "gyro_ts"].view("int64")) / 1e6)
C:\Users\shali\AppData\Local\Temp\ipykernel_68520\4049600805.py:19: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  delta_ms = ((merged.loc[have_gyro, "timestamp"].view("int64") - merged.loc[have_gyro, "gyro_ts"].view("int64")) / 1e6)


,timestamp,Ax,Ay,Az,Gx,Gy,Gz
0,2025-03-24 07:52:19+00:00,"[""-62.48186"",""-27.60828"",""-95.41802"",""-155.478...","[""-758.6971"",""-984.1958"",""-1240.532"",""-1338.82...","[""-1097.585"",""-1221.784"",""-1086.742"",""-957.614...","[""39.15614"",""32.96125"",""22.04125"",""1.041086"",""...","[""-7.520049"",""-10.77502"",""-9.620019"",""-4.79006...","[""7.881836"",""7.742001"",""12.78200"",""2.981754"",""..."
1,2025-03-24 07:52:20+00:00,"[""-178.7271"",""-161.2903"",""-133.1977"",""-124.479...","[""-1360.991"",""-1342.681"",""-1269.442"",""-1171.14...","[""-777.2302"",""-619.5170"",""-460.8181"",""-345.490...","[""2.160977"",""1.005977"",""1.531018"",""1.986018"",""...","[""3.259907"",""3.924907"",""3.679918"",""4.589918"",""...","[""-10.80841"",""-11.50841"",""-10.73835"",""-9.47834..."
2,2025-03-24 07:52:20+00:00,"[""-81.85607"",""-92.51189"",""-101.2303"",""-112.854...","[""-1140.310"",""-1056.471"",""-967.8134"",""-805.917...","[""-254.8053"",""-147.3632"",""-53.72105"",""-31.0497...","[""2.160977"",""1.005977"",""1.531018"",""1.986018"",""...","[""3.259907"",""3.924907"",""3.679918"",""4.589918"",""...","[""-10.80841"",""-11.50841"",""-10.73835"",""-9.47834..."


## 7) Resample to exact 50 Hz (gap-free)

#### What/why
Make a perfect 50 Hz grid (20 ms), use mean per bin, then interpolate tiny gaps. Recreate timestamp_ms.

In [24]:
# ---- edit here if needed ----
TARGET_HZ = 50
# -----------------------------

step_ms = int(round(1000 / TARGET_HZ))
freq_str = f"{step_ms}ms"

# Ensure numeric dtypes for sensor cols
for c in ["Ax","Ay","Az","Gx","Gy","Gz"]:
    merged[c] = pd.to_numeric(merged[c], errors="coerce")

fixed = (
    merged.set_index("timestamp")[["Ax","Ay","Az","Gx","Gy","Gz"]]  # numeric only
          .resample(freq_str).mean()
          .interpolate(limit_direction="both")
          .reset_index()
)

fixed["timestamp_ms"] = (fixed["timestamp"].view("int64") // 10**6).astype("int64")

print(f"[Step 7] Fixed @ {TARGET_HZ} Hz:", fixed.shape)
fixed.head(5)


[Step 7] Fixed @ 50 Hz: (181701, 8)


C:\Users\shali\AppData\Local\Temp\ipykernel_68520\1704905874.py:19: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  fixed["timestamp_ms"] = (fixed["timestamp"].view("int64") // 10**6).astype("int64")


,timestamp,Ax,Ay,Az,Gx,Gy,Gz,timestamp_ms
0,2025-03-24 07:52:19+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1742802739000
1,2025-03-24 07:52:19.020000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1742802739020
2,2025-03-24 07:52:19.040000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1742802739040
3,2025-03-24 07:52:19.060000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1742802739060
4,2025-03-24 07:52:19.080000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1742802739080


In [25]:
# ---- edit here if needed ----
USE_RAD_PER_SEC = False
# -----------------------------

if USE_RAD_PER_SEC:
    for c in ["Gx","Gy","Gz"]:
        fixed[c] = np.deg2rad(fixed[c])

fixed = fixed[["timestamp","timestamp_ms","Ax","Ay","Az","Gx","Gy","Gz"]]
print("[Step 8] Columns:", list(fixed.columns))
fixed.head(3)


[Step 8] Columns: ['timestamp', 'timestamp_ms', 'Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']


,timestamp,timestamp_ms,Ax,Ay,Az,Gx,Gy,Gz
0,2025-03-24 07:52:19+00:00,1742802739000,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-03-24 07:52:19.020000+00:00,1742802739020,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-03-24 07:52:19.040000+00:00,1742802739040,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
assert fixed["timestamp"].is_monotonic_increasing, "Timestamps are NOT sorted!"
dupes = fixed["timestamp"].duplicated().sum()
print(f"[Step 9] Duplicate timestamps: {dupes}")

assert np.isfinite(fixed[["Ax","Ay","Az","Gx","Gy","Gz"]]).all().all(), "Found non-finite values!"

accel_ok = (fixed[["Ax","Ay","Az"]].abs() < 50).all().all()
gyro_ok  = (fixed[["Gx","Gy","Gz"]].abs() < 2000).all().all()
print(f"[Step 9] Accel magnitude OK: {accel_ok} | Gyro magnitude OK: {gyro_ok}")

duration_s = (fixed["timestamp"].iloc[-1] - fixed["timestamp"].iloc[0]).total_seconds()
print(f"[Step 9] Duration: {duration_s:.2f}s, Rows: {len(fixed)}, Target Hz: {TARGET_HZ}")


[Step 9] Duplicate timestamps: 0


AssertionError: Found non-finite values!



# 0) Notebook title & overview

**Markdown**

> # Garmin Wearable — Simple Accel+Gyro Processing (Easy Mode)
>
> **Goal:** turn two raw Excel files (accelerometer & gyroscope) with batched samples into a single, clean **50 Hz** time series with columns:
> `timestamp, timestamp_ms, Ax, Ay, Az, Gx, Gy, Gz`.
>
> **Key ideas:**
>
> 1. Parse list-like columns → 2) explode to one row per sample → 3) build true timestamps → 4) align accel↔gyro by nearest time → 5) resample to exact 50 Hz.
>
> We keep everything **UTC**, avoid clever abstractions, and comment every step.

---

# 1) Imports & file paths

**Code**

```python
# 1) Imports & simple paths (edit the two paths below)
import pandas as pd
import numpy as np
import json, ast

# >>> EDIT THESE TWO LINES ONLY FOR YOUR FILES <<<
ACCEL_PATH = "2025-03-23-15-23-10-accelerometer_data.xlsx"
GYRO_PATH  = "2025-03-23-15-23-10-gyroscope_data.xlsx"

# for resampling/alignment (edit if needed)
TARGET_HZ = 50                 # resample target (Hz)
MERGE_TOLERANCE_MS = 5         # max accel↔gyro time gap allowed
```

---

# 2) Load raw Excel & peek

**Markdown**

> We read both Excel files as-is to see the original column names.
> Many Garmin exports have:
>
> * `timestamp` (date string)
> * `timestamp_ms` (base ms offset)
> * `sample_time_offset` (list of per-sample ms within the row)
> * per-axis arrays like `calibrated_accel_x|y|z` or already `x|y|z`.

**Code**

```python
accel_raw = pd.read_excel(ACCEL_PATH)
gyro_raw  = pd.read_excel(GYRO_PATH)

print("ACCEL cols:", list(accel_raw.columns))
print("GYRO  cols:", list(gyro_raw.columns))
display(accel_raw.head(2))
display(gyro_raw.head(2))
```

---

# 3) Normalize column names to a simple schema

**Markdown**

> To make later steps easy, we normalize headers:
>
> * Accelerometer → `x, y, z`
> * Gyroscope     → `x, y, z`
>   (We’ll add the `A`/`G` prefixes **after** exploding.)

**Code**

```python
# For accel
accel_map = {
    "calibrated_accel_x": "x",
    "calibrated_accel_y": "y",
    "calibrated_accel_z": "z"
}
for k,v in accel_map.items():
    if k in accel_raw.columns:
        accel_raw = accel_raw.rename(columns={k:v})

# For gyro
gyro_map = {
    "calibrated_gyro_x": "x",
    "calibrated_gyro_y": "y",
    "calibrated_gyro_z": "z"
}
for k,v in gyro_map.items():
    if k in gyro_raw.columns:
        gyro_raw = gyro_raw.rename(columns={k:v})

# Check we have the basics we need
needed = ["timestamp","timestamp_ms","sample_time_offset","x","y","z"]
missing_acc = [c for c in needed if c not in accel_raw.columns]
missing_gyr = [c for c in needed if c not in gyro_raw.columns]
print("Missing in accel:", missing_acc)
print("Missing in gyro :", missing_gyr)
```

---

# 4) Convert list-like cells into real Python lists

**Markdown**

> In the raw files, `sample_time_offset` and `x/y/z` may be stored as JSON strings (like `"["0","20","40"]"`) or Python-like lists (like `"[0, 20, 40]"`).
> We parse them into real lists so we can **explode** them.

**Code**

```python
def parse_list_cell(v):
    """Return a Python list from JSON/Python-like representations."""
    if isinstance(v, (list, tuple, np.ndarray)):
        return list(v)
    if pd.isna(v):
        return []
    s = str(v).strip()
    # Try JSON first
    try:
        return json.loads(s)
    except Exception:
        # Then Python literal (safe) e.g. "[0, 20, 40]"
        try:
            return ast.literal_eval(s)
        except Exception:
            # Fallback: try comma-split
            s = s.strip("[]")
            if not s:
                return []
            return [pd.to_numeric(x, errors="coerce") for x in s.split(",")]

# Apply to both dataframes
for df in (accel_raw, gyro_raw):
    for c in ["sample_time_offset", "x", "y", "z"]:
        df[c] = df[c].apply(parse_list_cell)

print("Parsed list lengths (first row accel):",
      [len(accel_raw.iloc[0][c]) for c in ["sample_time_offset","x","y","z"]])
print("Parsed list lengths (first row gyro):",
      [len(gyro_raw.iloc[0][c]) for c in ["sample_time_offset","x","y","z"]])
```

---

# 5) Drop any rows where list lengths are mismatched

**Markdown**

> Each row’s `offsets`, `x`, `y`, and `z` must have the **same length**.
> If not, we drop that row (rare, but safer than guessing).

**Code**

```python
def ok_lengths(row):
    a = len(row["sample_time_offset"])
    b = len(row["x"]); c = len(row["y"]); d = len(row["z"])
    return (a == b == c == d)

acc_ok = accel_raw.apply(ok_lengths, axis=1)
gyr_ok = gyro_raw.apply(ok_lengths, axis=1)

print("Accel bad rows:", (~acc_ok).sum(), " | Gyro bad rows:", (~gyr_ok).sum())

accel_raw = accel_raw.loc[acc_ok].reset_index(drop=True).copy()
gyro_raw  = gyro_raw.loc[gyr_ok].reset_index(drop=True).copy()
```

---

# 6) Explode to **one row per sample** (accel & gyro)

**Markdown**

> We convert batched rows into **per-sample** rows.
> Then we create **true timestamps** as:
> `timestamp_true = to_datetime(timestamp, UTC) + timestamp_ms (ms) + sample_time_offset (ms)`.

**Code**

```python
# Explode in-place (pandas can explode multiple columns at once)
accel_long = accel_raw.explode(["sample_time_offset","x","y","z"], ignore_index=True).copy()
gyro_long  = gyro_raw.explode (["sample_time_offset","x","y","z"], ignore_index=True).copy()

# Build true timestamps (UTC)
def make_true_time(df):
    # base = timestamp (UTC) + timestamp_ms
    base = pd.to_datetime(df["timestamp"], utc=True, errors="coerce") \
           + pd.to_timedelta(pd.to_numeric(df["timestamp_ms"], errors="coerce").fillna(0).astype("int64"), unit="ms")
    # add per-sample offset
    off  = pd.to_timedelta(pd.to_numeric(df["sample_time_offset"], errors="coerce").fillna(0).astype("int64"), unit="ms")
    return base + off

accel_long["timestamp"] = make_true_time(accel_long)
gyro_long["timestamp"]  = make_true_time(gyro_long)

# Keep only what we need, and rename axes with prefixes
accel_long = accel_long[["timestamp","x","y","z"]].rename(columns={"x":"Ax","y":"Ay","z":"Az"})
gyro_long  = gyro_long [["timestamp","x","y","z"]].rename(columns={"x":"Gx","y":"Gy","z":"Gz"})

# Ensure numeric types
for c in ["Ax","Ay","Az"]:
    accel_long[c] = pd.to_numeric(accel_long[c], errors="coerce")
for c in ["Gx","Gy","Gz"]:
    gyro_long[c]  = pd.to_numeric(gyro_long[c],  errors="coerce")

# Sort
accel_long = accel_long.sort_values("timestamp").reset_index(drop=True)
gyro_long  = gyro_long.sort_values("timestamp").reset_index(drop=True)

print("accel_long:", accel_long.shape, "gyro_long:", gyro_long.shape)
display(accel_long.head(3))
display(gyro_long.head(3))
```

---

# 7) Align accel & gyro by **nearest timestamp** (with tolerance)

**Markdown**

> For each accel sample, find the nearest gyro sample within a small time window.
> If there’s no gyro within the window, we drop that accel row.
> We also print **match ratio** and basic **lag** statistics.

**Code**

```python
tolerance = pd.Timedelta(milliseconds=MERGE_TOLERANCE_MS)

gyro_for_merge = gyro_long.rename(columns={"timestamp":"gyro_ts"})
merged = pd.merge_asof(
    accel_long.sort_values("timestamp"),
    gyro_for_merge.sort_values("gyro_ts"),
    left_on="timestamp", right_on="gyro_ts",
    direction="nearest", tolerance=tolerance
)

have_gyro = merged[["Gx","Gy","Gz"]].notna().all(axis=1)
match_ratio = have_gyro.mean()
lag_ms = ((merged.loc[have_gyro, "timestamp"].astype("int64")
          - merged.loc[have_gyro, "gyro_ts"].astype("int64")) / 1e6)

print(f"Matched within {MERGE_TOLERANCE_MS} ms: {match_ratio:.1%}")
if len(lag_ms):
    print(f"Lag ms — mean: {lag_ms.mean():.3f}, std: {lag_ms.std():.3f}, max_abs: {np.abs(lag_ms).max():.3f}")
else:
    print("No matched samples. Increase MERGE_TOLERANCE_MS?")

# Keep only matched rows and drop helper column
merged = merged.loc[have_gyro, ["timestamp","Ax","Ay","Az","Gx","Gy","Gz"]].reset_index(drop=True)
display(merged.head(5))
```

---

# 8) Resample to **exact 50 Hz** (gap-free)

**Markdown**

> We convert the aligned series into a perfect **50 Hz** timeline (every 20 ms).
> We aggregate with `mean` within bins and use time interpolation for tiny gaps.
> Finally we add `timestamp_ms` for convenience.

**Code**

```python
step_ms = int(round(1000 / TARGET_HZ))
freq = f"{step_ms}ms"

# Ensure numeric (protect against stray objects)
for c in ["Ax","Ay","Az","Gx","Gy","Gz"]:
    merged[c] = pd.to_numeric(merged[c], errors="coerce")

fixed = (merged
         .set_index("timestamp")[["Ax","Ay","Az","Gx","Gy","Gz"]]
         .resample(freq)
         .mean()
         .interpolate(method="time", limit_direction="both")
         .reset_index())

# Add timestamp_ms (int)
fixed["timestamp_ms"] = (fixed["timestamp"].astype("int64") // 10**6).astype("int64")

print("Fixed @", TARGET_HZ, "Hz:", fixed.shape)
display(fixed.head(5))
```

---

# 9) Sanity checks (order, duplicates, finites, rough ranges)

**Markdown**

> Quick guards to catch obvious problems early.

**Code**

```python
# monotonic timestamps
assert fixed["timestamp"].is_monotonic_increasing, "Timestamps not sorted!"

# duplicates
dupes = fixed["timestamp"].duplicated().sum()
print("Duplicate timestamps:", dupes)

# finite values
assert np.isfinite(fixed[["Ax","Ay","Az","Gx","Gy","Gz"]]).all().all(), "Non-finite values found!"

# rough magnitude checks (tweak if your units differ)
accel_ok = (fixed[["Ax","Ay","Az"]].abs() < 50).all().all()      # ~5g if m/s² (~49)
gyro_ok  = (fixed[["Gx","Gy","Gz"]].abs() < 2000).all().all()    # <2000 deg/s typical
print("Accel magnitude OK:", accel_ok, "| Gyro magnitude OK:", gyro_ok)

duration_s = (fixed["timestamp"].iloc[-1] - fixed["timestamp"].iloc[0]).total_seconds()
print(f"Duration: {duration_s:.2f}s | Rows: {len(fixed)} | Target Hz: {TARGET_HZ}")
```

---

# 10) Save outputs (Parquet + CSV)

**Markdown**

> Parquet is fast & typed; CSV is handy to eyeball in Excel.

**Code**

```python
# >>> EDIT THESE OUTPUT PATHS IF YOU WANT <<<
PARQUET_PATH = "processed/session_50hz.parquet"
CSV_PATH     = "processed/session_50hz.csv"

import os
os.makedirs(os.path.dirname(PARQUET_PATH), exist_ok=True)

fixed.to_parquet(PARQUET_PATH, index=False)
fixed.to_csv(CSV_PATH, index=False)

print("Saved:", PARQUET_PATH, "|", CSV_PATH)
```

---

## Why this is easier

* No functions except tiny **parse\_list\_cell** and **make\_true\_time**-style code; everything is **linear**.
* Each step has a **single purpose** and small edit points (paths, tolerance, Hz).
* We rely on **pandas explode** (clear + fast) instead of manual loops.
* Simple **diagnostics** (match %, lag) tell you when to tweak `MERGE_TOLERANCE_MS`.

## If you still hit errors

1. Run these quick prints right after the failing step and share the output:

```python
print(accel_raw.dtypes); print(accel_raw.head(2))
print(gyro_raw.dtypes);  print(gyro_raw.head(2))
print(accel_long.dtypes); print(accel_long.head(2))
print(gyro_long.dtypes);  print(gyro_long.head(2))
print(merged.dtypes);     print(merged.head(2))
```

2. Common fixes:

* Increase `MERGE_TOLERANCE_MS` from 5 → 10 or 20 if match ratio is low.
* If your pandas is old and can’t explode multiple columns at once, explode one by one:

  ```python
  accel_tmp = accel_raw.explode("sample_time_offset")
  accel_tmp["x"] = accel_tmp["x"].apply(lambda lst: lst.pop(0) if isinstance(lst, list) and len(lst)>0 else np.nan)
  # (or switch to a small loop like we used earlier)
  ```

Want me to package these cells into a polished `.ipynb` with headings already in place?


love it — let’s add **labels** and **windowing** the same “easy-mode” way.
No config objects. Tiny “EDIT HERE” lines. Clear markdown. Drop-in cells.

---

# 11) Add labels (time-window style)

**Markdown**

> We’ll tag each row in `fixed` with a `label` based on **time windows**.
> Two simple options for your label windows file:
>
> 1. **JSON** list (recommended)
>
>    ```json
>    [
>      {"label":"rest", "start_ms":1711185600000, "end_ms":1711185660000},
>      {"label":"walk", "start_ms":1711185660000, "end_ms":1711185780000}
>    ]
>    ```
>
>    (times in **milliseconds since epoch**; inclusive start, exclusive end)
> 2. **CSV** with header: `label,start_iso,end_iso` (ISO 8601 timestamps, UTC)
>
>    ```
>    rest, 2025-03-23T10:00:00Z, 2025-03-23T10:01:00Z
>    walk, 2025-03-23T10:01:00Z, 2025-03-23T10:03:00Z
>    ```
>
> We’ll support both. Pick one by editing a variable.

**Code**

```python
import os, json
import pandas as pd
import numpy as np

# ====== EDIT HERE ======
LABEL_SOURCE = "json"      # "json" or "csv"
JSON_PATH    = "labels/session_001.json"
CSV_PATH     = "labels/session_001.csv"
# =======================

# Prepare an empty label column first
fixed["label"] = "unknown"

# Helper: apply time windows to 'fixed'
def apply_label_windows_from_ms(fixed_df, windows, default="unknown"):
    # windows: list of dicts with label, start_ms, end_ms (end is exclusive)
    tsms = fixed_df["timestamp_ms"].values
    lbl  = np.array([default]*len(fixed_df), dtype=object)
    for w in windows:
        s = int(w["start_ms"]); e = int(w["end_ms"])
        mask = (tsms >= s) & (tsms < e)
        lbl[mask] = w["label"]
    fixed_df = fixed_df.copy()
    fixed_df["label"] = lbl
    return fixed_df

def read_windows():
    if LABEL_SOURCE == "json" and os.path.exists(JSON_PATH):
        with open(JSON_PATH, "r") as f:
            L = json.load(f)
        # basic validation
        for w in L:
            assert "label" in w and ("start_ms" in w or "start" in w)
        # Support optional ISO keys too
        out = []
        for w in L:
            if "start_ms" in w and "end_ms" in w:
                out.append({"label": w["label"], "start_ms": int(w["start_ms"]), "end_ms": int(w["end_ms"])})
            else:
                # if ISO strings provided
                s = pd.Timestamp(w["start"], tz="UTC").value // 10**6
                e = pd.Timestamp(w["end"],   tz="UTC").value // 10**6
                out.append({"label": w["label"], "start_ms": int(s), "end_ms": int(e)})
        return out

    if LABEL_SOURCE == "csv" and os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
        assert {"label","start_iso","end_iso"}.issubset(df.columns), "CSV needs columns: label,start_iso,end_iso"
        out = []
        for _, r in df.iterrows():
            s = pd.to_datetime(r["start_iso"], utc=True).value // 10**6
            e = pd.to_datetime(r["end_iso"],   utc=True).value // 10**6
            out.append({"label": r["label"], "start_ms": int(s), "end_ms": int(e)})
        return out

    print("⚠️ No labels file found. Keeping 'unknown' for all rows.")
    return []

windows = read_windows()
if len(windows):
    fixed = apply_label_windows_from_ms(fixed, windows, default="unknown")

fixed["label"].value_counts(dropna=False).head()
```

**What you’ll get**

* `fixed` now has a `label` column.
* Prints the class counts (unknown + any applied labels).

---

# 12) Sliding windowing (build model inputs)

**Markdown**

> We’ll convert the 50 Hz stream into **overlapping windows** for ML/DL:
>
> * Window length = `WINDOW_SEC` seconds
> * Hop (stride)   = `HOP_SEC` seconds
> * Features per sample: `[Ax, Ay, Az, Gx, Gy, Gz]`
> * Window label = **majority label** inside the window
>
>   * We’ll drop windows where the majority label coverage is below a threshold (e.g. 80%), or if the majority is `"unknown"`.

**Code**

```python
import numpy as np
import pandas as pd
from collections import Counter

# ====== EDIT HERE ======
WINDOW_SEC          = 2.0   # e.g., 2-second windows
HOP_SEC             = 1.0   # e.g., 50% overlap at 50 Hz
ASSUMED_HZ          = 50    # our 'fixed' is at 50 Hz
MIN_LABEL_COVERAGE  = 0.80  # require at least 80% of samples in a window to be the majority label
DROP_UNKNOWN_WINDOWS = True # drop windows whose majority is "unknown"
FEATURE_COLS = ["Ax","Ay","Az","Gx","Gy","Gz"]
# =======================

# Safety: ensure order and no NaNs in features
fixed = fixed.sort_values("timestamp").reset_index(drop=True)
for c in FEATURE_COLS:
    fixed[c] = pd.to_numeric(fixed[c], errors="coerce")
fixed[FEATURE_COLS] = fixed[FEATURE_COLS].interpolate(limit_direction="both")
assert np.isfinite(fixed[FEATURE_COLS].values).all(), "Non-finite values remain after interpolation."

win_len = int(round(WINDOW_SEC * ASSUMED_HZ))
hop_len = int(round(HOP_SEC * ASSUMED_HZ))
assert win_len > 0 and hop_len > 0, "Window/Hop must be >= 1 sample."

X_list, y_list, t_start_list, t_end_list = [], [], [], []
labels_arr = fixed["label"].values
feat_arr   = fixed[FEATURE_COLS].values
ts_arr     = fixed["timestamp"].values

N = len(fixed)
i = 0
while i + win_len <= N:
    window_feats  = feat_arr[i:i+win_len]
    window_labels = labels_arr[i:i+win_len]
    # majority label
    c = Counter(window_labels)
    majority_label, maj_count = c.most_common(1)[0]
    coverage = maj_count / win_len

    # rules to keep/drop
    if coverage >= MIN_LABEL_COVERAGE and (not DROP_UNKNOWN_WINDOWS or majority_label != "unknown"):
        X_list.append(window_feats.copy())
        y_list.append(majority_label)
        t_start_list.append(ts_arr[i])
        t_end_list.append(ts_arr[i+win_len-1])

    i += hop_len

X = np.array(X_list, dtype=np.float32)          # shape: (num_windows, win_len, 6)
y_labels = np.array(y_list, dtype=object)       # shape: (num_windows,)
t_bounds = np.array(list(zip(t_start_list, t_end_list)), dtype=object)

print("Windows created:", X.shape, "| kept labels:", len(y_labels))
print("Label distribution:", Counter(y_labels))
print("Window length (samples):", win_len, "| Hop (samples):", hop_len)
```

**What you’ll get**

* `X`: `(num_windows, win_len, 6)` float32 array
* `y_labels`: `(num_windows,)` array of string labels
* `t_bounds`: start/end timestamps per window

---

# 13) Encode labels to integers (optional but handy)

**Markdown**

> DL frameworks prefer integer class IDs. We’ll map label strings → integers.

**Code**

```python
# Build mapping from seen labels
unique_labels = sorted(list(set(y_labels.tolist())))
label_to_id = {lbl:i for i,lbl in enumerate(unique_labels)}
id_to_label = {i:lbl for lbl,i in label_to_id.items()}

y = np.array([label_to_id[lbl] for lbl in y_labels], dtype=np.int64)
print("label_to_id:", label_to_id)
print("X:", X.shape, "| y:", y.shape)
```

**What you’ll get**

* `y`: integer labels aligned with `X`
* `label_to_id` / `id_to_label` dicts

---

# 14) Quick train/val/test split (stratified-ish)

**Markdown**

> We’ll do a simple randomized split that keeps class balance roughly intact.

**Code**

```python
import numpy as np

# ====== EDIT HERE ======
TRAIN_FRAC = 0.7
VAL_FRAC   = 0.15
TEST_FRAC  = 0.15
RANDOM_SEED = 7
# =======================

assert abs(TRAIN_FRAC + VAL_FRAC + TEST_FRAC - 1.0) < 1e-9

rng = np.random.default_rng(RANDOM_SEED)
idx = np.arange(len(X))
rng.shuffle(idx)

n = len(idx)
n_train = int(TRAIN_FRAC * n)
n_val   = int(VAL_FRAC   * n)
train_idx = idx[:n_train]
val_idx   = idx[n_train:n_train+n_val]
test_idx  = idx[n_train+n_val:]

X_train, y_train = X[train_idx], y[train_idx]
X_val,   y_val   = X[val_idx],   y[val_idx]
X_test,  y_test  = X[test_idx],  y[test_idx]

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)
```

---

# 15) Save windowed dataset (NPZ)

**Markdown**

> Save a compact `.npz` you can load in PyTorch/TF or scikit-learn later.

**Code**

```python
# ====== EDIT HERE ======
NPZ_PATH = "processed/session_windows_2s_1s_50hz.npz"
# =======================

os.makedirs(os.path.dirname(NPZ_PATH), exist_ok=True)
np.savez_compressed(
    NPZ_PATH,
    X=X, y=y,
    X_train=X_train, y_train=y_train,
    X_val=X_val,     y_val=y_val,
    X_test=X_test,   y_test=y_test,
    label_to_id=json.dumps(label_to_id),
    id_to_label=json.dumps(id_to_label),
    feature_names=json.dumps(FEATURE_COLS),
    window_sec=WINDOW_SEC, hop_sec=HOP_SEC, hz=ASSUMED_HZ
)
print("Saved:", NPZ_PATH)
```

---

## Sanity recap (what you have now)

* `fixed`: clean **50 Hz** stream with `Ax..Gz` + `label` per row
* `X, y_labels, y`: windowed inputs + labels (majority vote)
* Train/Val/Test splits + a saved **.npz** bundle

---

## Tips you can tweak anytime

* **Overlap & window size:** change `WINDOW_SEC` / `HOP_SEC`.
* **Coverage rule:** raise/lower `MIN_LABEL_COVERAGE` to be stricter/looser.
* **Unknown handling:** set `DROP_UNKNOWN_WINDOWS=False` if you want to keep them (often better to drop).
* **Unit conversion:** if your gyro is deg/s and a model expects rad/s, toggle earlier (Step 8 in the base pipeline).

If you want, I can add a tiny **feature standardization** cell (fit on train, apply to val/test) and a **PyTorch dataset** snippet next.
